# Packages

In [1]:
import torch, os, sys, warnings,tqdm, importlib
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sodtool.sodt_utils as sodt_utils
import sodtool.evaluator as lhtevaluator
from PIL import Image
warnings.filterwarnings("ignore")

In [2]:
importlib.reload(sodt_utils)
importlib.reload(lhtevaluator)

<module 'sodtool.evaluator' from '/home/hxl170008/code/NSF/paper_redundancy/sodtool/evaluator.py'>

# Configuration

In [3]:
class Config:
    root = r'/data/lht/data/NDT_Deer'
    exp_name = "backup/FasterRCNN"
    roi_best = 0.5
    num_acc  = 0.5
    areaRng = [[0, 10000000000.0], [0,187],[187.5,405],[408,10000000000.0]]
    metric = ['AP_0.5:0.95_all','AP_0.5_all','AP_0.75_all','AP_0.5:0.95_small','AP_0.5:0.95_medium','AP_0.5:0.95_large',
          'AR_0.5:0.95_all_MD1','AR_0.5:0.95_all_MD10','AR_0.5:0.95_all_MD100',
          'AR_0.5:0.95_small','AR_0.5:0.95_medium','AR_0.5:0.95_large',
         ]
#
opt = Config()
if os.path.isdir(opt.exp_name):
    print(opt.exp_name)
else:
    os.mkdir(opt.exp_name)

backup/FasterRCNN


# Model

In [4]:
from torchvision.models.detection.rpn import AnchorGenerator

In [5]:
fpn_resnets={}
resnet_weights=sodt_utils.resnet_weights
resnet_names = sodt_utils.resnet_names

for name in tqdm.tqdm(resnet_names):
    weights = resnet_weights[name]
    fpn_resnets[name] = sodt_utils.lht_ResNetFPNBackBoneByLayerID(resnet_name=name,weights=weights,trainable_layers=5,returned_layers=[1,2,3,4])

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 5/5 [00:04<00:00,  1.10it/s]


In [6]:
frcnn_fpn_res = {}
aspect_ratios = ((0.5, 1.0, 2.0),) * 5
featmap_names=['0','1','2','3']
anchor_sizes = ((4,8,16,32,64),)*5
roi_output_size=(8,10)
num_classes = 2
# fpn based on COCO datasets
for k,v in fpn_resnets.items():
    rpn_anchor_generator =  AnchorGenerator(anchor_sizes, aspect_ratios)
    m = sodt_utils.lhtInitializeFasterRCNN_FPN_ResNet(rpn_anchor_generator=rpn_anchor_generator, roi_output_size=roi_output_size,
                                                 featmap_names=featmap_names, bb_res_fpn=v, num_classes=num_classes)
    frcnn_fpn_res[k] = m

# Train

In [7]:
fp1 = os.path.join(opt.root, 'Train','imgs.txt')
fp2 = os.path.join(opt.root, 'Train','anns.txt')
trainImgs = np.loadtxt(fp1,dtype=str)
trainAnns = np.loadtxt(fp2,dtype=str)
#
fp1 = os.path.join(opt.root, 'Test','imgs.txt')
fp2 = os.path.join(opt.root, 'Test','anns.txt')
testImgs = np.loadtxt(fp1,dtype=str)
testAnns = np.loadtxt(fp2,dtype=str)
#
trainDS = sodt_utils.BBoxDataset(root=opt.root,imgs=trainImgs,anns=trainAnns,transform=sodt_utils.get_transform())
testDS  = sodt_utils.BBoxDataset(root=opt.root,imgs=testImgs,anns=testAnns, transform=sodt_utils.get_transform())
#
train_loader = torch.utils.data.DataLoader(trainDS,batch_size=5,shuffle=True, num_workers=1, collate_fn=sodt_utils.collate_fn)
test_loader  = torch.utils.data.DataLoader(testDS,batch_size=5,shuffle=False, num_workers=1, collate_fn=sodt_utils.collate_fn)
f'train:{len(trainDS)}, test: {len(testDS)}'

'train:1833, test: 444'

In [8]:
fp1 = os.path.join(opt.root, 'TrainGIF','imgs.txt')
fp2 = os.path.join(opt.root, 'TrainGIF','anns.txt')
trainImgs = np.loadtxt(fp1,dtype=str)
trainAnns = np.loadtxt(fp2,dtype=str)
#
fp1 = os.path.join(opt.root, 'TestGIF','imgs.txt')
fp2 = os.path.join(opt.root, 'TestGIF','anns.txt')
testImgs = np.loadtxt(fp1,dtype=str)
testAnns = np.loadtxt(fp2,dtype=str)
#
trainDS_gif = sodt_utils.BBoxDataset(root=opt.root,imgs=trainImgs,anns=trainAnns,transform=sodt_utils.get_transform())
testDS_gif  = sodt_utils.BBoxDataset(root=opt.root,imgs=testImgs,anns=testAnns,transform=sodt_utils.get_transform())
#
train_gif_loader  = torch.utils.data.DataLoader(trainDS_gif,batch_size=2,shuffle=False, num_workers=1, collate_fn=sodt_utils.collate_fn)
test_gif_loader  = torch.utils.data.DataLoader(testDS_gif,batch_size=2,shuffle=False, num_workers=1, collate_fn=sodt_utils.collate_fn)
f'train:{len(trainDS_gif)}, test:{len(testDS_gif)}'

'train:59, test:30'

In [9]:
#model = FasterRCNN_VGG16_BN()
dic_gif_train, dic_gif_test = {},{}
ret_bratios={}
device = torch.device("cuda")
num_epochs = 500
step_size_up = 300 #50
patience = 20

In [ ]:
if hasattr(tqdm,'notebook'):
    pr = tqdm.notebook.tqdm(frcnn_fpn_res.items())
else:
    pr = tqdm.tqdm_notebook(frcnn_fpn_res.items())
for k,frcnn_fpn in pr:
    fn = os.path.join(opt.exp_name,'NDT_Deer',k)
    if not os.path.isdir(fn):
        os.mkdir(fn)
    palette_id = trainDS.pid
    fn = os.path.join(fn,str(palette_id))
    if not os.path.isdir(fn):
        os.mkdir(fn)
    suffix_name = f'deer_fpn_{k}'
    print(f'palette: {palette_id}, model: {k}, path:{fn}, suffix:{suffix_name}')
    #
    gif_train, gif_test = [],[]
    # initialize model parameters
    frcnn_fpn.to(device)
    params = [p for p in frcnn_fpn.parameters() if p.requires_grad]
    optimizer = torch.optim.SGD(params, lr=0.01)
    lr_scheduler = torch.optim.lr_scheduler.CyclicLR(optimizer,base_lr=0.0001,max_lr=0.01,step_size_up=step_size_up,mode='triangular2')
    bratios = np.zeros(12)
    coco_stats = []
    global_avg_losses=[]
    # train
    stale=0
    num_iter = tqdm.notebook.tqdm(range(0,num_epochs), leave=False)
    for epoch in num_iter:
        logger=sodt_utils.lht_train_one_epoch(frcnn_fpn,optimizer,train_loader,device,epoch,print_freq=len(train_loader),warm=True)
        lr_scheduler.step()
        lr = optimizer.param_groups[0]['lr']
        num_iter.set_postfix(lr=lr)
        coco_evaluator = lhtevaluator.lht_get_coco_evaluator(frcnn_fpn,test_loader,device=device,areaRng=opt.areaRng)
        stats = lhtevaluator.lht_get_coco_stats(coco_evaluator)
        coco_stats.append(stats.tolist())
        #losses
        global_avg_losses.append(logger.loss.global_avg)
        tratios = stats
        ids = np.argwhere(tratios>bratios).ravel().tolist()
        if len(ids) == 0:
            stale += 1
            if stale > patience:
                print(f"No improvment {patience} consecutive epochs, early stopping")
                break
        else:
            stale = 0
            preds_test = sodt_utils.lht_predictByDSLoader(test_gif_loader, device, frcnn_fpn)        
            preds_train = sodt_utils.lht_predictByDSLoader(train_gif_loader, device, frcnn_fpn)
            gif_test.append(preds_test)
            gif_train.append(preds_train)
            print(f'stats:{stats}')
        for i in ids:
            fn_m = f"{suffix_name}_{i}.ckpt"
            fp = os.path.join(fn,fn_m)
            torch.save(frcnn_fpn.state_dict(), fp)
            bratios[i] = tratios[i]
    #
    # save coco_stats.npy
    coco_fp = os.path.join(fn,"coco_stats.npy")
    with open(coco_fp, 'wb') as fin:
        np.save(fin, np.array(coco_stats))
    # save global_avg_losses.npy
    loss_fp = os.path.join(fn,"global_avg_losses.npy")
    with open(loss_fp, 'wb') as fin:
        np.save(fin, np.array(global_avg_losses))
    # save the model in the last iteration.
    fn_m = f"{suffix_name}_end.ckpt"
    fp = os.path.join(fn,fn_m)
    torch.save(frcnn_fpn.state_dict(), fp)
    #
    ret_bratios[k] = bratios
    dic_gif_test[k] = gif_test
    dic_gif_train[k] = gif_train

  0%|          | 0/5 [00:00<?, ?it/s]

palette: 0, model: resnet18, path:backup/FasterRCNN/NDT_Deer/resnet18/0, suffix:deer_fpn_resnet18


  0%|          | 0/500 [00:00<?, ?it/s]

  0%|          | 0/367 [00:00<?, ?it/s]

Epoch: [0] [  0/367] eta: 0:12:12 lr: 0.000100 loss: 1.2786 (1.2786) time: 1.9971 data: 0.4779 max mem: 1663
Epoch: [0] [366/367] eta: 0:00:00 lr: 0.000001 loss: 0.2831 (0.5153) time: 0.1918 data: 0.0470 max mem: 2536
Epoch: [0] Total time: 0:01:10 (0.1931 s / it)
creating index...
index created!


...

  0%|          | 0/15 [00:00<?, ?it/s]

  0%|          | 0/30 [00:00<?, ?it/s]

stats:[0.33140878 0.78229912 0.18226553 0.25280941 0.33341301 0.39564216
 0.09276286 0.35065999 0.47897132 0.42582897 0.50662055 0.48300654]


  0%|          | 0/367 [00:00<?, ?it/s]

Epoch: [11] [  0/367] eta: 0:02:09 lr: 0.000463 loss: 0.2375 (0.2375) time: 0.3525 data: 0.1808 max mem: 2556


In [20]:
sodt_utils.lht_get_cuda_info()

id: 0, name: NVIDIA GeForce RTX 3090
Memory Usage
Allocated: 0.0 GB
Max memory reserved:  0.0 GB
Total memory:  23.7 GB
Free memory:  23.7 GB
--------------------
